In [ ]:
from Graph import Vertex
from Graph import Graph
import helpers
import random
# 0 is left bucket
#1 is right bucket
class FM:
    def __init__(self,allowedPasses,graph,partition) -> None:
        self.g=graph
        self.leftBucket={}
        self.rightBucket={}
        self.partition=partition
        self.lockedVertices=[]
        self.solutions=[]
        self.allowedPasses=allowedPasses

    def getPart(self,id,partition):
        return partition[id-1]

    def calculateCost(self,vertex,partition):
        cost=0
        part=self.getPart(vertex.id,partition)
        for neigbour in vertex.neighbours:
            neigbourPart=self.getPart(neigbour,partition)
            if neigbourPart!=part:
                cost+=1
        return cost

    def calculateGain(self,vertex,partition):
        cost=self.calculateCost(vertex,partition)
        return cost-(vertex.numNeighbours-cost)
    
    def getMaxCost(self):
        maxcost=-1
        for v in self.g.vertices:
            cost = v.numNeighbours
            if maxcost<cost:
                maxcost=cost
        return maxcost

    def initializaBuckets(self,maxCost):
        self.rightBucket = {i: set() for i in range(maxCost, -maxCost - 1, -1)}
        self.leftBucket = {i: set() for i in range(maxCost, -maxCost - 1, -1)}

    def fillBuckets(self):
        for v in self.g.vertices:
            part=self.getPart(v.id,self.partition)
            gain=self.calculateGain(v,self.partition)
            if (part==0):
                self.leftBucket[gain].add(v.id)
            else:
                self.rightBucket[gain].add(v.id)
    def areBucketsEmpty(self):
        for (_,values) in self.leftBucket.items():
            if len(values)>0:
                return False
        for (_,values) in self.rightBucket.items():
             if len(values)>0:
                return False
        return True
    
    def pickVertex(self,bucket):
        for (key,values) in bucket.items():
            if len(values)!=0:
                vertices=list(bucket[key])
                random.shuffle(vertices)
                value = vertices[0]
                self.lockedVertices.append(value)
                bucket[key].remove(vertices[0])
                return value
            
    def moveVertex(self,src):
        prevPartition=self.partition.copy()
        prevGains=[]
        newGains=[]
        if (src==0):
            vertexId=self.pickVertex(self.leftBucket)
            self.partition[vertexId-1]=1
            vertex=self.getVertex(vertexId)
            for n in vertex.neighbours:
                if n not in self.lockedVertices:
                    prevGain = self.calculateGain(self.getVertex(n),prevPartition)
                    newGain = self.calculateGain(self.getVertex(n),self.partition)
                    prevGains.append((n,prevGain))
                    newGains.append((n,newGain))
        else:
            vertexId=self.pickVertex(self.rightBucket)
            self.partition[vertexId-1]=0
            vertex=self.getVertex(vertexId)
            for n in vertex.neighbours:
                if n not in self.lockedVertices:
                    prevGain = self.calculateGain(self.getVertex(n),prevPartition)
                    newGain = self.calculateGain(self.getVertex(n),self.partition)
                    prevGains.append((n,prevGain))
                    newGains.append((n,newGain))
        self.alterBuckets(prevGains,newGains)
        
    def getVertex(self,id):
        return self.g.vertices[id-1]
    
    def alterBuckets(self,prevGains,newGains):
        for ((v,prevGain),(v,newGain)) in zip(prevGains,newGains):
            part=self.getPart(v,self.partition)
            if (part==0):
                self.leftBucket[prevGain].remove(v)
                self.leftBucket[newGain].add(v)
            else:
                self.rightBucket[prevGain].remove(v)
                self.rightBucket[newGain].add(v)
    
    def calculateFitness(self,partition):
        res=0
        for v in self.g.vertices:
            res+=self.calculateCost(v,partition)
        return res/2
    
    def pickBestSolution(self):
        minCut=1000000
        bestSolution=None
        for (cut,solution) in self.solutions:
            if minCut>cut:
                minCut=cut
                bestSolution=solution
        return (minCut,bestSolution)
    
    def FM_pass(self):
        self.solutions=[]
        self.lockedVertices=[]
        src=0
        counter=0
        while len(self.lockedVertices)!=500:
            self.moveVertex(src)
            if src==0:
                src=1
            else:
                src=0
            counter+=1
            if counter==2:
                partCopy=self.partition.copy()
                fitness = self.calculateFitness(partCopy)
                self.solutions.append((fitness,partCopy))
                counter=0
        bestSoltution=self.pickBestSolution() 
        fitness = self.calculateFitness(bestSoltution[1])
        return ((fitness,bestSoltution[1]))
    def FM_run(self):
        solutions=[]
        solutions.append((self.calculateFitness(self.partition),self.partition))
        for _ in range (0,self.allowedPasses):
            maxCost=self.getMaxCost()
            self.initializaBuckets(maxCost)
            self.fillBuckets()
            (minCut,partition)=self.FM_pass()
            solutions.append((minCut,partition.copy()))
            self.partition=partition
        return (minCut,self.partition)



In [ ]:
from FM import FM
import helpers

class MLS:
    def __init__(self,graph,timesToRestart) -> None:
        self.timesToRestart=timesToRestart
        self.graph=graph
        self.FMPassesAllowed=int(10000/timesToRestart)
        self.solutions=[]
    def pickBestSolution(self):
        minCut=1000
        bestSolution=-1
        for (f,s) in self.solutions:
            if f<minCut:
                bestSolution=(f,s)
                minCut=f
        return bestSolution
    def MLS_run (self):
        for i in range(0,self.timesToRestart):
            partition=helpers.createPartition()
            fm=FM(graph=self.graph,allowedPasses=self.FMPassesAllowed,partition=partition)
            (fitness,solution)=fm.FM_run()
            print (i, (fitness,solution))
            self.solutions.append((fitness,solution))
           # print(self.solutions)
        bestSolution=self.pickBestSolution()
        return bestSolution


In [ ]:
import random

def createPartition():
    length = 250
    partition = [0] * length + [1] * length
    random.shuffle(partition)
    return partition

In [ ]:
from List import Node
from List import DoubleLinkedList
from Graph import Graph
from MLS import MLS
from GLS import GLS
from ILS import ILS
import time

timesToRestartMLS=120
glsFMPassesPerChild=60
glsPopulationSize=50
glsSolutions=[]
mlsSolutions=[]

def main():
    t0 = time.time()
    g=Graph()
    g.serealize()
   
    for _ in range (0,1):
        mls=MLS(g,timesToRestartMLS)
        mlsSolutions.append(mls.MLS_run())
        #gls=GLS(g,glsPopulationSize,glsFMPassesPerChild)
       # glsSolutions.append(gls.runGLS())
    t1 = time.time()
    totalTime = t1-t0  
    print(totalTime)
main()
    

In [ ]:
class Vertex:
     def __init__(self):
        self.id=-1
        self.coordinates=(None,None)
        self.neighbours=[]
        self.numNeighbours=-1
def remove_spaces(splitLines):
    res = [i for i in splitLines if i != '']
    res = [i for i in res if i != '\n']
    return res
class Graph:
    def __init__(self):
        self.vertices=[]
    def serealize(self):
        file1 = open("graphData.txt", 'r')
        Lines = file1.readlines()
        # Strips the newline character
        for line in Lines:
            v=Vertex()
            splitLine=line.split(' ')
            splitLine=remove_spaces(splitLine)
            id=int(splitLine[0])
            coordinates=splitLine[1]
            v.numNeighbours=int(splitLine[2])
            neighbours=[]
            for i in range(3,len(splitLine)):
               neighbours.append(int(splitLine[i]))
            v.id=id
            v.neighbours=neighbours
            v.coordinates=coordinates
            self.vertices.append(v)

               


